In [2]:
# init
from pyquda.utils import gauge_utils
from pyquda.field import LatticeFermion
from pyquda.enum_quda import QudaParity
from pyquda import init, core, quda, pyqcu, mpi, pointer
import os
import sys
from time import perf_counter
import cupy as cp
test_dir = os.path.dirname(os.path.abspath("./"))
sys.path.insert(0, os.path.join(test_dir, ".."))
os.environ["QUDA_RESOURCE_PATH"] = ".cache"
latt_size = [16, 32, 32, 64]
grid_size = [1, 1, 1, 1]
Lx, Ly, Lz, Lt = latt_size
Nd, Ns, Nc = 4, 4, 3
Gx, Gy, Gz, Gt = grid_size
latt_size = [Lx//Gx, Ly//Gy, Lz//Gz, Lt//Gt]
_latt_size = [Lx//Gx//2, Ly//Gy, Lz//Gz, Lt//Gt]
Lx, Ly, Lz, Lt = latt_size
Vol = Lx * Ly * Lz * Lt
mpi.init(grid_size)
latt_shape = (Lt, Lz, Ly, Lx//2, Ns, Nc)
param = pyqcu.QcuParam()
param.lattice_size = latt_size
dslash = core.getDslash(latt_size, -3.5, 0, 0, anti_periodic_t=False)
kappa = 0.125
U = gauge_utils.gaussGauge(latt_size, 0)
dslash.loadGauge(U)

Disabling GPU-Direct RDMA access
Enabling peer-to-peer copy engine and direct load/store access
QUDA 1.1.0 (git 1.1.0--sm_80)
CUDA Driver version = 12040
CUDA Runtime version = 12030
Found device 0: NVIDIA GeForce RTX 4060 Laptop GPU
 -- This might result in a lower performance. Please consider adjusting QUDA_GPU_ARCH when running cmake.

Using device 0: NVIDIA GeForce RTX 4060 Laptop GPU
Loaded 103 sets of cached parameters from .cache/tunecache.tsv
Loaded 103 sets of cached parameters from .cache/tunecache.tsv
cublasCreated successfully
Creating Gaussian distrbuted Lie group field with sigma = 1.000000e-01


In [3]:
# give ans first
ans_e = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_e
ans_o = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_o
print("## ans_o = ", ans_o[0, 0, 0, 0, 0, 0])

## ans_o =  (0.7061157568259795+0.0433102981616878j)


In [4]:
# give x_o, b__o, r, r_tilde, p, v, s, t, latt_tmp0, latt_tmp1
x_o = cp.zeros(latt_shape, cp.complex128)
b_e = cp.zeros(latt_shape, cp.complex128)
b_o = cp.zeros(latt_shape, cp.complex128)
b__o = cp.zeros(latt_shape, cp.complex128)
r = cp.zeros(latt_shape, cp.complex128)
r_tilde = cp.zeros(latt_shape, cp.complex128)
p = cp.zeros(latt_shape, cp.complex128)
s = cp.zeros(latt_shape, cp.complex128)
t = cp.zeros(latt_shape, cp.complex128)
latt_tmp0 = cp.zeros(latt_shape, cp.complex128)
latt_tmp1 = cp.zeros(latt_shape, cp.complex128)
zero = cp.zeros(latt_shape, cp.complex128)
# give r_norm2, MAX_ITER, TOL, rho_prev, rho, alpha, omega, beta, tmp0, tmp1, kappa
r_norm2 = 0
MAX_ITER = 1e2
TOL = 1e-6
rho_prev = 1
rho = 0
alpha = 1
omega = 1
beta = 0
tmp0 = 0
tmp1 = 0
kappa = 0.125

In [5]:
# define dslash
def dslash(src_o, dest_o):
    latt_tmp0 = zero
    latt_tmp1 = zero
    _latt_tmp0 = LatticeFermion(_latt_size, latt_tmp0)
    _latt_tmp1 = LatticeFermion(_latt_size, latt_tmp1)
    _src_o = LatticeFermion(_latt_size, src_o)
    pyqcu.dslashQcu(_latt_tmp0.even_ptr, _src_o.even_ptr,
                    U.data_ptr, param, 0)  # D_eo
    pyqcu.dslashQcu(_latt_tmp1.even_ptr, _latt_tmp0.even_ptr,
                    U.data_ptr, param, 1)  # D_oe
    return src_o-kappa**2*latt_tmp1

In [6]:
# give b'_o(b__0)
_latt_tmp0 = LatticeFermion(_latt_size, latt_tmp0)
_latt_tmp1 = LatticeFermion(_latt_size, latt_tmp1)
_ans_e = LatticeFermion(_latt_size, ans_e)
_ans_o = LatticeFermion(_latt_size, ans_o)
_b_e = LatticeFermion(_latt_size, b_e)
_b_o = LatticeFermion(_latt_size, b_o)
latt_tmp0 = zero
pyqcu.dslashQcu(_latt_tmp0.even_ptr, _ans_o.even_ptr,
                U.data_ptr, param, 0)  # give D_eo ans_o
b_e = ans_e-kappa*latt_tmp0
latt_tmp1 = zero
pyqcu.dslashQcu(_latt_tmp1.even_ptr, _ans_e.even_ptr,
                U.data_ptr, param, 1)  # give D_oe ans_e
b_o = ans_o-kappa*latt_tmp1
latt_tmp1 = zero
pyqcu.dslashQcu(_latt_tmp1.even_ptr, _b_e.even_ptr,
                U.data_ptr, param, 1)  # give D_oe b_e
b__o = b_o+kappa*latt_tmp1

In [11]:
def dot(a, b):
    return cp.inner(a.flatten().conjugate(), b.flatten())

In [7]:
# bistabcg
r = b__o
r_tilde = r
for i in range(1, int(MAX_ITER)):
    rho = cp.dot(r_tilde, r)
    beta = (rho / rho_prev) * (alpha / omega)
    p = r + (p - v * omega) * beta
    # v = A * p
    dslash(p, v)
    alpha = rho / cp.dot(r_tilde, v)
    s = r - v * alpha
    # t = A * s
    dslash(s, t)
    omega = cp.dot(t, s) / cp.dot(t, t)
    x_o = x_o + p * alpha + s * omega
    r = s - t * omega
    r_norm2 = cp.dot(r, r)
    # break
    if (r_norm2 < TOL or i == MAX_ITER - 1):
        print("## turns:", i)
        break
    rho_prev = rho

print('## difference: ', cp.linalg.norm(x_o - ans_o) / cp.linalg.norm(ans_o))

ValueError: Axis dimension mismatch

In [8]:
print(r.shape)

(64, 32, 32, 8, 4, 3)


In [9]:
print(r_tilde.shape)

(64, 32, 32, 8, 4, 3)


In [10]:
cp.dot(r_tilde, r)

ValueError: Axis dimension mismatch